In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = "13173023" # 店舗ID
SPREADSHEET_KEY = '1wsGjhNvoUAlsWnpSq_IusoV_cZAjJ3iiObgCslhaIX8' # スプレッドシートのID
pstart = 1 # パチンコの1台目
#sstart = 385 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p16row = worksheet.cell(11,  25).value
p1row = worksheet.cell(11,  24).value
p5row = worksheet.cell(11,  26).value
#s20row = worksheet.cell(11,  28).value
#s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p16table = []
p1table = []
p5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "160":
                    p16table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "50":
                    p5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "160":
                        p16table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "50":
                        p5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass


wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1.6RAW'!F" + str(int(p16row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p16table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.5RAW'!F" + str(int(p5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p5table})
#wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
#wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/18', '025677', '1', '9487.0', '13543.0', '15', '5']
['08/18', '025677', '2', '11538.0', '-9027.0', '1', '1']
['08/18', '025677', '3', '14359.0', '-10231.0', '2', '2']
['08/18', '025677', '4', '14359.0', '-6619.0', '5', '3']
['08/18', '025677', '5', '14359.0', '-11735.0', '1', '1']
['08/18', '025677', '6', '12051.0', '1806.0', '9', '2']
['08/18', '025677', '7', '12821.0', '-902.0', '7', '3']
['08/18', '025826', '8', '0', '0', '0', '0']
['08/18', '025826', '9', '0', '0', '0', '0']
['08/18', '025826', '10', '0', '0', '0', '0']
['08/18', '025826', '11', '0', '0', '0', '0']
['08/18', '025818', '12', '0', '0', '0', '0']
['08/18', '025818', '13', '1026.0', '-1203.0', '0', '0']
['08/18', '025818', '14', '4103.0', '2709.0', '4', '1']
['08/18', '025869', '15', '4103.0', '9029.0', '12', '1']
['08/18', '025869', '16', '1026.0', '-1203.0', '0', '0']
['08/18', '025888', '17', '17179.0', '-3008.0', '8', '3']
['08/18', '025888', '18', '513.0', '904.0', '1', '1']
['08/18', '025888', '19', '26923.0

['08/18', '025895', '149', '15897.0', '-902.0', '10', '2']
['08/18', '025895', '150', '27692.0', '-1203.0', '19', '6']
['08/18', '025895', '151', '26923.0', '32200.0', '41', '6']
['08/18', '025895', '152', '43077.0', '6621.0', '32', '6']
['08/18', '025895', '153', '31795.0', '22871.0', '34', '3']
['08/18', '025895', '154', '26154.0', '-3309.0', '14', '4']
['08/18', '025895', '155', '34103.0', '2709.0', '23', '5']
['08/18', '025895', '156', '34615.0', '-29490.0', '4', '4']
['08/18', '025895', '157', '35385.0', '-20462.0', '7', '2']
['08/18', '025895', '158', '45897.0', '-28888.0', '14', '9']
['08/18', '025849', '159', '513.0', '-902.0', '0', '0']
['08/18', '025849', '160', '5641.0', '-3911.0', '1', '1']
['08/18', '025849', '161', '0', '0', '0', '0']
['08/18', '025864', '162', '1026.0', '-902.0', '0', '0']
['08/18', '025864', '163', '4615.0', '-902.0', '6', '1']
['08/18', '025859', '164', '4615.0', '-2707.0', '1', '1']
['08/18', '025859', '165', '1026.0', '-902.0', '0', '0']
['08/18', '0

['08/18', '025699', '295', '6154.0', '-4814.0', '1', '1']
['08/18', '025636', '296', '8974.0', '3311.0', '11', '1']
['08/18', '025675', '297', '14359.0', '-2707.0', '12', '6']
['08/18', '025737', '298', '22821.0', '-12638.0', '12', '3']
['08/18', '025364', '299', '13077.0', '-3610.0', '10', '5']
['08/18', '025655', '300', '33846.0', '-15647.0', '33', '11']
['08/18', '025369', '301', '22564.0', '-2106.0', '49', '17']
['08/18', '025145', '302', '16410.0', '-2707.0', '12', '6']
['08/18', '025626', '303', '7949.0', '-902.0', '14', '14']
['08/18', '025626', '304', '9231.0', '-6920.0', '2', '2']
['08/18', '025235', '305', '20256.0', '-4814.0', '19', '7']
['08/18', '025670', '306', '21538.0', '1806.0', '14', '3']
['08/18', '025670', '307', '18462.0', '-5115.0', '6', '1']
['08/18', '025859', '308', '18718.0', '603.0', '27', '1']
['08/18', '025779', '309', '18205.0', '-4814.0', '13', '4']
['08/18', '025847', '310', '7692.0', '-6920.0', '2', '2']
['08/18', '025847', '311', '14615.0', '-6920.0', 

['08/18', '025533', '435', '20513.0', '-2106.0', '22', '7']
['08/18', '025533', '436', '34359.0', '-3309.0', '41', '11']
['08/18', '025506', '437', '18718.0', '904.0', '26', '8']
['08/18', '025466', '438', '16410.0', '-6319.0', '9', '4']
['08/18', '025572', '439', '22564.0', '904.0', '28', '8']
['08/18', '025572', '440', '31795.0', '-2106.0', '36', '11']
['08/18', '025579', '441', '4359.0', '-902.0', '2', '1']
['08/18', '025635', '442', '10513.0', '-4814.0', '4', '1']
['08/18', '025737', '443', '9231.0', '2107.0', '18', '1']
['08/18', '025489', '444', '12564.0', '-6619.0', '2', '1']
['08/18', '025489', '445', '20769.0', '-3008.0', '8', '2']
['08/18', '025636', '446', '513.0', '-902.0', '0', '0']
['08/18', '025636', '447', '22308.0', '-12939.0', '7', '2']
['08/18', '025636', '448', '10256.0', '-3309.0', '5', '2']
['08/18', '025364', '449', '15641.0', '-5717.0', '11', '5']
['08/18', '025364', '450', '7949.0', '904.0', '11', '4']
['08/18', '025364', '451', '20000.0', '-6920.0', '14', '7']

{'spreadsheetId': '1wsGjhNvoUAlsWnpSq_IusoV_cZAjJ3iiObgCslhaIX8',
 'updates': {'spreadsheetId': '1wsGjhNvoUAlsWnpSq_IusoV_cZAjJ3iiObgCslhaIX8',
  'updatedRange': 'P0.5RAW!F4952:L4981',
  'updatedRows': 30,
  'updatedColumns': 7,
  'updatedCells': 210}}